<a href="https://colab.research.google.com/github/sipocz/pid_time_series/blob/main/model_3d_2023_03_17/pid_NN_3d_20230317_201000.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import pandas as pd
import os
import seaborn as sns
import wandb

In [3]:
from google.colab import drive

In [4]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
#user = "Anna"
user = "SL"
uzem = "Szint3"
data_source="SINT3_415/3D_transposed"
#fname="72C03_TC_error_toNN.csv"
fname_good = "415_SC_3D_part"
fname_bad = "415_SC_3D_part"
fname_good_ext=["1.csv","3.csv","5.csv","7.csv"]
fname_bad_ext=["2.csv","4.csv","6.csv","8.csv"]


In [6]:
# Elérési út a 415_SC_error-hoz
if user=="Anna":
    path_good = "/content/drive/MyDrive/Egyetem_MSc/Diplomamunka/2022Anna/Datapipeline/" + data_source + "/" + fname_good
    path_bad = "/content/drive/MyDrive/Egyetem_MSc/Diplomamunka/2022Anna/Datapipeline/" + data_source + "/" + fname_bad
    path_fig = "/content/drive/MyDrive/Egyetem_MSc/Diplomamunka/2022Anna/Datapipeline/plots/"
else:
    path_good = "/content/drive/MyDrive/2022Anna/Datapipeline/" + data_source + "/" + fname_good 
    path_bad = "/content/drive/MyDrive/2022Anna/Datapipeline/" + data_source + "/" + fname_bad
    path_fig = "/content/drive/MyDrive/2022Anna/Datapipeline/plots/"


In [7]:
print(path_good)
print(path_bad)

/content/drive/MyDrive/2022Anna/Datapipeline/SINT3_415/3D_transposed/415_SC_3D_part
/content/drive/MyDrive/2022Anna/Datapipeline/SINT3_415/3D_transposed/415_SC_3D_part


In [8]:
cols=[str(i) for i in range(60)]

In [9]:
df_good=pd.DataFrame(columns=cols)
df_bad=pd.DataFrame(columns=cols)
print(df_good.head())
for i,goods in enumerate(fname_good_ext):
    df_good_tmp = pd.read_csv(path_good+goods,usecols=None )
    df_good_tmp.columns=cols
    df_good=pd.concat([df_good,df_good_tmp],axis=0,)
    df_bad_tmp = pd.read_csv(path_bad+fname_bad_ext[i],usecols=None)
    df_bad_tmp.columns=cols
    df_bad=pd.concat([df_bad,df_bad_tmp],axis=0)

Empty DataFrame
Columns: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59]
Index: []

[0 rows x 60 columns]


In [10]:
print(df_good.isnull().values.any())
print(df_bad.isnull().values.any())

False
False


In [11]:
df_good.tail()

,0,1,2,3,4,5,6,7,8,9,...,50,51,52,53,54,55,56,57,58,59
569,45.0,45.0,45.0,45.0,45.0,45.0,45.0,45.0,45.0,45.0,...,0.216297,0.216297,0.216297,0.216297,0.216297,0.216297,0.216297,0.216297,0.216297,0.216297
570,45.0,45.0,45.0,45.0,45.0,45.0,45.0,45.0,45.0,45.0,...,0.216297,0.216297,0.216297,0.216297,0.216297,0.216297,0.216297,0.216297,0.216297,0.216297
571,45.0,45.0,45.0,45.0,45.0,45.0,45.0,45.0,45.0,45.0,...,0.216297,0.216297,0.216297,0.216297,0.216297,0.216297,0.216297,0.216297,0.216297,0.216297
572,45.0,45.0,45.0,45.0,45.0,45.0,45.0,45.0,45.0,45.0,...,0.216297,0.216297,0.216297,0.216297,0.216297,0.216297,0.216297,0.216297,0.216297,0.163654
573,45.0,45.0,45.0,45.0,45.0,45.0,45.0,45.0,45.0,45.0,...,0.216297,0.216297,0.216297,0.216297,0.216297,0.216297,0.216297,0.216297,0.163654,0.216297


In [12]:

import matplotlib.pyplot as plot

df_ = df_good

# You must normalize the data before applying the fit method
df_good_normalized=(df_ - df_.mean()) / df_.std()

# Normalize bad data with the good data parameters
df_bad_normalized=(df_bad - df_.mean()) / df_.std()

In [13]:
df_good_normalized["state"]=0
df_bad_normalized["state"]=1

In [14]:
df_good_normalized=df_good_normalized.reindex()

In [15]:
df_good_normalized

,0,1,2,3,4,5,6,7,8,9,...,51,52,53,54,55,56,57,58,59,state
0,-1.453455,-1.454954,-1.456455,-1.457960,-1.459467,-1.460977,-1.462489,-1.464005,-1.465523,-1.467043,...,-0.024124,-0.024124,-0.024124,-0.024124,-0.024124,-0.024124,-0.024124,-0.024118,-0.024223,0
1,-1.453455,-1.454954,-1.456455,-1.457960,-1.459467,-1.460977,-1.462489,-1.464005,-1.465523,-1.467043,...,-0.024124,-0.024124,-0.024124,-0.024124,-0.024124,-0.024124,-0.024124,-0.024118,-0.024223,0
2,-1.453455,-1.454954,-1.456455,-1.457960,-1.459467,-1.460977,-1.462489,-1.464005,-1.465523,-1.467043,...,-0.024124,-0.024124,-0.024124,-0.024124,-0.024124,-0.024124,-0.024124,-0.024118,-0.024223,0
3,-1.453455,-1.454954,-1.456455,-1.457960,-1.459467,-1.460977,-1.462489,-1.464005,-1.465523,-1.467043,...,-0.024124,-0.024124,-0.024124,-0.024124,-0.024124,-0.024124,-0.024124,-0.024118,-0.024223,0
4,-1.453455,-1.454954,-1.456455,-1.457960,-1.459467,-1.460977,-1.462489,-1.464005,-1.465523,-1.467043,...,-0.024124,-0.024124,-0.024124,-0.024124,-0.024124,-0.024124,-0.024124,-0.024118,-0.024223,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
569,-0.691638,-0.692917,-0.694198,-0.695481,-0.696765,-0.698051,-0.699339,-0.700628,-0.701918,-0.703211,...,-0.025480,-0.025480,-0.025480,-0.025480,-0.025480,-0.025479,-0.025479,-0.025473,-0.025578,0
570,-0.691638,-0.692917,-0.694198,-0.695481,-0.696765,-0.698051,-0.699339,-0.700628,-0.701918,-0.703211,...,-0.025480,-0.025480,-0.025480,-0.025480,-0.025480,-0.025479,-0.025479,-0.025473,-0.025578,0
571,-0.691638,-0.692917,-0.694198,-0.695481,-0.696765,-0.698051,-0.699339,-0.700628,-0.701918,-0.703211,...,-0.025480,-0.025480,-0.025480,-0.025480,-0.025480,-0.025479,-0.025479,-0.025473,-0.025578,0
572,-0.691638,-0.692917,-0.694198,-0.695481,-0.696765,-0.698051,-0.699339,-0.700628,-0.701918,-0.703211,...,-0.025480,-0.025480,-0.025480,-0.025480,-0.025480,-0.025479,-0.025479,-0.025473,-0.041168,0


In [16]:
df_all_normalized=pd.concat([df_good_normalized,df_bad_normalized],axis=0)

In [17]:
df_all_normalized

,0,1,2,3,4,5,6,7,8,9,...,51,52,53,54,55,56,57,58,59,state
0,-1.453455,-1.454954,-1.456455,-1.457960,-1.459467,-1.460977,-1.462489,-1.464005,-1.465523,-1.467043,...,-0.024124,-0.024124,-0.024124,-0.024124,-0.024124,-0.024124,-0.024124,-0.024118,-0.024223,0
1,-1.453455,-1.454954,-1.456455,-1.457960,-1.459467,-1.460977,-1.462489,-1.464005,-1.465523,-1.467043,...,-0.024124,-0.024124,-0.024124,-0.024124,-0.024124,-0.024124,-0.024124,-0.024118,-0.024223,0
2,-1.453455,-1.454954,-1.456455,-1.457960,-1.459467,-1.460977,-1.462489,-1.464005,-1.465523,-1.467043,...,-0.024124,-0.024124,-0.024124,-0.024124,-0.024124,-0.024124,-0.024124,-0.024118,-0.024223,0
3,-1.453455,-1.454954,-1.456455,-1.457960,-1.459467,-1.460977,-1.462489,-1.464005,-1.465523,-1.467043,...,-0.024124,-0.024124,-0.024124,-0.024124,-0.024124,-0.024124,-0.024124,-0.024118,-0.024223,0
4,-1.453455,-1.454954,-1.456455,-1.457960,-1.459467,-1.460977,-1.462489,-1.464005,-1.465523,-1.467043,...,-0.024124,-0.024124,-0.024124,-0.024124,-0.024124,-0.024124,-0.024124,-0.024118,-0.024223,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
515,-0.310730,-0.311899,-0.313070,-0.314242,-0.315414,-0.316588,-0.317763,-0.318939,-0.320116,-0.321295,...,-0.370495,2.344881,3.525656,1.690100,-2.078618,-2.669683,0.108053,2.199832,2.951400,1
516,-0.310730,-0.311899,-0.313070,-0.314242,-0.315414,-0.316588,-0.317763,-0.318939,-0.320116,-0.321295,...,2.344881,3.525656,1.690100,-2.078618,-2.669683,0.108053,2.199826,2.951541,3.221172,1
517,-0.310730,-0.311899,-0.313070,-0.314242,-0.315414,-0.316588,-0.317763,-0.318939,-0.320116,-0.321295,...,3.525656,1.690100,-2.078618,-2.669683,0.108053,2.199826,2.951536,3.221317,1.193817,1
518,-0.310730,-0.311899,-0.313070,-0.314242,-0.315414,-0.316588,-0.317763,-0.318939,-0.320116,-0.321295,...,1.690100,-2.078619,-2.669684,0.108052,2.199826,2.951536,3.221312,1.193936,-2.382354,1


In [18]:
import wandb



In [81]:
_N1_=1100 #70  #700
_N2_=118 #12  #120
_lr_=0.001
_batch_size_=32
_drop1_=0.5
_drop2_=0.7
_epochs_=9500


In [82]:
config = {"lr": _lr_, "batch_size": _batch_size_,"architecture": "NN", 
          "depth": 2,
          "layer1":_N1_,  "layer2":_N2_, 
          "drop1":_drop1_,"drop2":_drop2_,
          "epochs":_epochs_
          
          
          }

wandb.init(project="pid_3d", entity="sipoczlaszlo",config=config)


epoch/accuracy,▁▂▄▄▄▅▅▅▅▆▆▆▆▆▇▇▆▇▇▇▇▇▇▇▇▇██▇███████████
epoch/epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
epoch/learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█████████████████
epoch/loss,█▆▅▄▄▄▄▄▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁
epoch/lr,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█████████████████
epoch/val_accuracy,▁▃▄▄▄▅▆▅▆▇▇▇▇▇▇▇▇███████████████████████
epoch/val_loss,█▆▅▅▄▄▄▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/accuracy,0.85808
epoch/epoch,869
epoch/learning_rate,0.002
epoch/loss,0.32055


In [83]:
from keras.engine.base_layer import regularizers
from keras.layers import InputLayer, Dense, LSTM, Input, Dropout
from keras.models import Sequential, Model
from keras.optimizers import SGD,Adam,Adamax,Nadam,Ftrl,Adadelta
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint
from keras.backend import clear_session
from tensorflow.keras.losses import mean_absolute_percentage_error, huber,kld
from sklearn.model_selection import train_test_split
from keras.callbacks import LearningRateScheduler

clear_session()

kernel_reg_1=tf.keras.regularizers.L2(0.1)

input_size=60


input1=Input(shape=(input_size,))
l1_out=Dense(_N1_,activation="swish",kernel_initializer='glorot_uniform',)(input1) # kernel_initializer='lecun_normal'
l2_out=Dropout(_drop1_)(l1_out)


l3_out=Dense(_N2_,activation="swish",kernel_initializer='glorot_uniform',)(l2_out) #kernel_initializer='lecun_normal',
l4_out=Dropout(_drop2_)(l3_out)

pred=Dense(1, activation="sigmoid",)(l4_out)

model = Model(inputs=input1, outputs=pred)
optimizer=Adamax(learning_rate=_lr_,) #

model.compile(loss='binary_crossentropy',
    optimizer=optimizer,
    metrics=["accuracy"])

In [84]:
# 35 5 1 relu relu sigmoid SGD 0.01 loss: 0.1402 - accuracy: 0.9435 - val_loss: 0.7302 - val_accuracy: 0.8548
# 35 12 1 relu relu sigmoid SGD 0.01 loss 0.1162 94.6% test : 85%
# 17 5 1 relu relu sigmoid SGD 0.01  loss: 0.1714 - accuracy: 0.9300 - val_loss: 0.9535 - val_accuracy: 0.8503
# 35 5 1 relu relu sigmoid Adam 0.01 loss: 0.1238 - accuracy: 0.9467 - val_loss: 5.7545 - val_accuracy: 0.8653
# 35 5 1 relu relu sigmoid Adamax 0.01 loss: 0.1184 - accuracy: 0.9525 - val_loss: 3.5327 - val_accuracy: 0.8428
# 35 5 1 relu relu sigmoid Adamax 0.001 loss: 0.1185 - accuracy: 0.9525 - val_loss: 2.3218 - val_accuracy: 0.8593
# 35 5 1 relu relu sigmoid Adamax 0.001 loss: 0.1041 - accuracy: 0.9576 - val_loss: 5.1465 - val_accuracy: 0.8353  +1300 epoch 
# 135 15 1 swish swish sigmoid Adamax 0.001 batch size:1 epoch 100 loss: 0.1707 - accuracy: 0.9352 - val_loss: 0.8066 - val_accuracy: 0.8892   **** egész jó



In [85]:
prediktorok=cols
X_NN=df_all_normalized[prediktorok][:]  # 
y_NN=df_all_normalized["state"][:]

In [86]:
model_file="model_PID__54_loss_0.116_vloss_0.115_acc_0.953_vacc_0.958.hdf5"
#model_file="model_PID__94_loss_0.116_vloss_0.115_acc_0.950_vacc_0.966.hdf5"
model_file="model_PID__4491_loss_0.115_vloss_0.679_acc_0.954_vacc_0.880.hdf5"

In [87]:
model_url="https://github.com/sipocz/pid_time_series/raw/main/model3/"+model_file

In [88]:
__load_file__=False

In [89]:
if __load_file__:
    ! rm *.hdf5 
    ! wget $model_url
    model.load_weights(model_file)

In [90]:
X_train,X_test,y_train,y_test=train_test_split(X_NN,y_NN,train_size=0.9,shuffle=True,random_state=33)

In [91]:
!rm *.hdf5 

In [92]:
def learning_rate_corrector(epoch,lr):
    if epoch > 4000:
        lr = 0.001
        return lr
    if epoch > 3000:
        lr = 0.002
        return lr
    if epoch > 2000:
        lr = 0.005
        return lr
    
    if epoch > 500:
        lr = 0.002
        return lr
    return lr
    

In [93]:
from wandb.keras import WandbMetricsLogger
fname="./model_PID_3D"
callbacks = [
        LearningRateScheduler(learning_rate_corrector,verbose=1),
        WandbMetricsLogger(),       
        ModelCheckpoint(filepath=fname+"_{epoch:04.0f}"+"_loss_{loss:.3f}_vloss_{val_loss:.3f}_acc_{accuracy:.3f}_vacc_{val_accuracy:.3f}.hdf5", monitor='loss',
                        verbose=2, save_best_only=True, mode='min')]


In [94]:
__learning__=True

In [95]:

if __learning__: 
    history = model.fit(X_train, y_train, epochs=_epochs_, batch_size=_epochs_, validation_data=(X_test, y_test),verbose=0,callbacks=callbacks,)

A streamkimeneten csak az utolsó 5000 sor látható.

Epoch 8251: LearningRateScheduler setting learning rate to 0.001.

Epoch 8251: loss did not improve from 0.08408

Epoch 8252: LearningRateScheduler setting learning rate to 0.001.

Epoch 8252: loss did not improve from 0.08408

Epoch 8253: LearningRateScheduler setting learning rate to 0.001.

Epoch 8253: loss did not improve from 0.08408

Epoch 8254: LearningRateScheduler setting learning rate to 0.001.

Epoch 8254: loss did not improve from 0.08408

Epoch 8255: LearningRateScheduler setting learning rate to 0.001.

Epoch 8255: loss did not improve from 0.08408

Epoch 8256: LearningRateScheduler setting learning rate to 0.001.

Epoch 8256: loss did not improve from 0.08408

Epoch 8257: LearningRateScheduler setting learning rate to 0.001.

Epoch 8257: loss did not improve from 0.08408

Epoch 8258: LearningRateScheduler setting learning rate to 0.001.

Epoch 8258: loss did not improve from 0.08408

Epoch 8259: LearningRateScheduler se

In [96]:
blabla

NameError: ignored

In [ ]:
__load_file__=True
model_file="model_PID__0634_loss_0.086_vloss_1.253_acc_0.961_vacc_0.886.hdf5"
model_url="https://github.com/sipocz/pid_time_series/raw/main/model3/"+model_file

In [ ]:
if __load_file__:
    ! rm *.hdf5 
    ! wget $model_url
    model.load_weights(model_file)

In [ ]:
y_pred=model.predict(X_test)

In [ ]:
# https://scikit-learn.org/stable/auto_examples/miscellaneous/plot_display_object_visualization.html#sphx-glr-auto-examples-miscellaneous-plot-display-object-visualization-py

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import Binarizer
from sklearn.metrics import roc_curve,roc_auc_score
from sklearn.metrics import RocCurveDisplay
from sklearn.metrics import ConfusionMatrixDisplay

In [ ]:
bina_transformer=Binarizer(threshold=0.5)
y_pred_transform=bina_transformer.fit_transform(y_pred)

In [ ]:
cm=confusion_matrix(y_test,y_pred_transform)


In [ ]:
cm_display = ConfusionMatrixDisplay(cm).plot()

In [ ]:
fpr, tpr, _ = roc_curve( y_pred_transform,y_test,pos_label=1)
roc_display = RocCurveDisplay(fpr=fpr, tpr=tpr).plot()
auc = roc_auc_score(y_test, y_pred_transform)

In [ ]:
df_all_normalized

In [ ]:
def color_changer(arr):
    o=["r" if i>0.5 else "g" for i in arr]
    return o

In [ ]:
def plotgraf(df_in, predicted):
    xkoordinata=[i for i in range(len(df_in["0"]))]
    plot.figure(figsize=(12,6))
    col_ch=color_changer(predicted)
    plot.scatter(xkoordinata,df_in["0"],c=col_ch,marker=".",alpha=0.3)
    plot.ylabel('értékek')
    plot.xlabel('index')
    plot
    plot.show()

In [ ]:
df_y_all_predict

In [ ]:
df_y_all_predict=model.predict(df_all_normalized[prediktorok])

In [ ]:
df_y_predict=df_y_all_predict.round()

In [ ]:
df_y_predict

In [ ]:
plotgraf(df_all_normalized[400:600],df_y_predict[400:600])

In [ ]:
auc

In [ ]:
def grafikon3(fx,desc1,txt1,desc2="",txt2="",desc3="",txt3="",ngraf=2,c1='rgba(35,128,132,0.8)', c2='rgba(193,99,99,0.8)',c3='rgba(193,99,99,0.8)',title=None):
    '''
    fx: dataFrame
    desc1:column1
    txt1: label1
    desc2:column2
    txt2: label2
    ngraf: number of graph
    c1: color1
    c2: color2
    title: graph title
    '''
    
    #x_=[i for i in range(len(y_pred))]
    if title==None:
      title=txt1+" "+txt2
    import plotly.express as px
    import plotly.graph_objects as go
    from plotly.subplots import make_subplots
    fig0 = make_subplots(specs=[[{"secondary_y": True}]])


    if ngraf>=3:
        fig0.add_trace(
            go.Bar(x=fx.index, y=fx[desc3], marker_color='rgba(225, 20, 20,0.2)',  name=txt3, showlegend=True, ),
              secondary_y=False,
            #row=1, col=1
        )


    if ngraf>=2:
        fig0.add_trace(
            go.Scatter(x=fx.index, y=fx[desc2], name=txt2, line=dict(color=c2) ,showlegend=True  ),
            secondary_y=False,
            #row=1, col=1

        )

    fig0.add_trace(
        go.Scatter(x=fx.index, y=fx[desc1], name=txt1, line=dict(color=c1) ,showlegend=True  ),
        secondary_y=False,
        #row=1, col=1

    )

    fig0.update_layout(
        title=title,
        autosize=False,
        width=1200,
        height=600,
        
        )

    print(title)
    fig0.update_yaxes(title_text="<b>"+title+"</b>", secondary_y=False)
    #fig0.update_yaxes(title_text="<b>Alarm státusz</b>", secondary_y=True)
    fig0.update_layout(paper_bgcolor='rgb(200,200,200)')
    fig0.show()

In [ ]:

history_df=pd.DataFrame({"epoch":history.epoch, "loss":history.history["loss"],"val_loss":history.history["val_loss"]})

In [ ]:
grafikon3(history_df,"loss","Loss","val_loss","Val_Loss",title=None)